In [ ]:
import asyncio
import aiohttp
import nest_asyncio
import pandas as pd
import os
import re
from tqdm.asyncio import tqdm
import time

# Allow nested asyncio in Jupyter if needed
nest_asyncio.apply()

# Function to fetch a single page with retries in case of failure
async def fetch_page(session, url, headers, retries=3):
    attempt = 0
    while attempt < retries:
        try:
            async with session.get(url, headers=headers) as response:
                if response.status == 200:
                    return await response.json()
                else:
                    print(f"Error: Failed to fetch {url}, Status code: {response.status}")
                    return None
        except Exception as e:
            print(f"Attempt {attempt + 1} failed for {url}: {e}")
            attempt += 1
            if attempt >= retries:
                print(f"Max retries reached for {url}. Skipping...")
                return None
            await asyncio.sleep(2)  # Wait before retrying

# Function to fetch all pages concurrently with batch processing
async def fetch_all_pages_concurrently(api_url, headers, batch_size=20):
    all_results = []
    next_url = api_url
    total_fetched = 0
    failed_urls = []  # List to store URLs that failed to fetch
    start_time = time.time()

    async with aiohttp.ClientSession() as session:
        with tqdm(desc="Fetching Results", unit="batch") as pbar:
            while next_url:
                print(f"Fetching: {next_url}")  # Debugging line
                data = await fetch_page(session, next_url, headers)
                if data and 'results' in data:
                    all_results.extend(data['results'])
                    next_url = data.get('next')  # Update the cursor
                    total_fetched += len(data['results'])
                else:
                    print(f"No results or failed request for {next_url}")
                    break

                pbar.set_postfix({"Fetched": total_fetched})  # Fixed this line to pass a dictionary
                pbar.update(len(data.get('results', [])))

                # Estimate the time remaining
                elapsed_time = time.time() - start_time
                estimated_time_remaining = (elapsed_time / total_fetched) * (len(data.get('results', [])) * 2)  # Just a heuristic for estimation
                print(f"Estimated Time Remaining: {estimated_time_remaining:.2f} seconds")

    end_time = time.time()
    print(f"\nFinished fetching data in {end_time - start_time:.2f} seconds.")
    print(f"Total records fetched: {total_fetched}")
    return all_results, failed_urls

# Function to clean and save the data
def process_and_save_data(all_results, save_path):
    # Create DataFrame
    df = pd.DataFrame(all_results)

    # Function to safely extract the first item from a list
    def safe_get_first(lst):
        return lst[0] if isinstance(lst, list) and len(lst) > 0 else None

    # Expand recap_documents if present
    if 'recap_documents' in df.columns:
        recap_docs = df['recap_documents'].apply(safe_get_first)
        recap_df = pd.json_normalize(recap_docs)
        recap_df.columns = ['recap_' + col for col in recap_df.columns]
        df = pd.concat([df.drop('recap_documents', axis=1), recap_df], axis=1)

    # Remove illegal characters
    ILLEGAL_CHARACTERS_RE = re.compile(r'[\000-\010]|[\013-\014]|[\016-\037]')
    df = df.applymap(lambda x: ILLEGAL_CHARACTERS_RE.sub('', x) if isinstance(x, str) else x)

    # Save to Excel or CSV based on availability
    try:
        df.to_excel(save_path, index=False)
        print(f"\nData saved successfully to {save_path}")
    except Exception as e:
        print(f"Error saving to Excel: {e}. Saving as CSV instead...")
        save_path_csv = save_path.replace('.xlsx', '.csv')
        df.to_csv(save_path_csv, index=False)
        print(f"Data saved successfully to {save_path_csv}")

# Main function to fetch and save data
async def main():
    api_url = "https://www.courtlistener.com/api/rest/v4/search/?q=%22Cybersecurity%22%20OR%20%22Data%20Privacy%22%20OR%20%22Data%20Security%22%20OR%20%22GDPR%22%20OR%20%22Data%20Protection%22%20OR%20%22Blockchain%22&type=r&order_by=dateFiled%20desc&filed_after=01%2F01%2F2024&filed_before=01%2F01%2F2025&court=scotus%20ca1%20ca2%20ca3%20ca4%20ca5%20ca6%20ca7%20ca8%20ca9%20ca10%20ca11%20cadc%20cafc%20dcd%20almd%20alnd%20alsd%20akd%20azd%20ared%20arwd%20cacd%20caed%20cand%20casd%20cod%20ctd%20ded%20flmd%20flnd%20flsd%20gamd%20gand%20gasd%20hid%20idd%20ilcd%20ilnd%20ilsd%20innd%20insd%20iand%20iasd%20ksd%20kyed%20kywd%20laed%20lamd%20lawd%20med%20mdd%20mad%20mied%20miwd%20mnd%20msnd%20mssd%20moed%20mowd%20mtd%20ned%20nvd%20nhd%20njd%20nmd%20nyed%20nynd%20nysd%20nywd%20nced%20ncmd%20ncwd%20ndd%20ohnd%20ohsd%20oked%20oknd%20okwd%20ord%20paed%20pamd%20pawd%20rid%20scd%20sdd%20tned%20tnmd%20tnwd%20txed%20txnd%20txsd%20txwd%20utd%20vtd%20vaed%20vawd%20waed%20wawd%20wvnd%20wvsd%20wied%20wiwd%20wyd%20gud%20nmid%20prd%20vid"
    headers = {"Authorization": "Token xyz"}

    # Fetch data using optimized batch fetching
    print("Fetching data...")
    all_results, failed_urls = await fetch_all_pages_concurrently(api_url, headers, batch_size=20)

    # Retry failed URLs if any
    if failed_urls:
        async with aiohttp.ClientSession() as session:
            retry_results = await retry_failed_urls(failed_urls, session, headers)
            all_results.extend(retry_results)

    # Define save path
    desktop_path = os.path.expanduser(r"C:\Users\91909\Desktop\AI")
    save_path = os.path.join(desktop_path, "q55.xlsx")

    # Process and save data
    process_and_save_data(all_results, save_path)

if __name__ == "__main__":
    asyncio.run(main())



Fetching data...


Fetching Results: 0batch [00:00, ?batch/s]

Fetching: https://www.courtlistener.com/api/rest/v4/search/?q=%22Cybersecurity%22%20OR%20%22Data%20Privacy%22%20OR%20%22Data%20Security%22%20OR%20%22GDPR%22%20OR%20%22Data%20Protection%22%20OR%20%22Blockchain%22&type=r&order_by=dateFiled%20desc&filed_after=01%2F01%2F2024&filed_before=01%2F01%2F2025&court=scotus%20ca1%20ca2%20ca3%20ca4%20ca5%20ca6%20ca7%20ca8%20ca9%20ca10%20ca11%20cadc%20cafc%20dcd%20almd%20alnd%20alsd%20akd%20azd%20ared%20arwd%20cacd%20caed%20cand%20casd%20cod%20ctd%20ded%20flmd%20flnd%20flsd%20gamd%20gand%20gasd%20hid%20idd%20ilcd%20ilnd%20ilsd%20innd%20insd%20iand%20iasd%20ksd%20kyed%20kywd%20laed%20lamd%20lawd%20med%20mdd%20mad%20mied%20miwd%20mnd%20msnd%20mssd%20moed%20mowd%20mtd%20ned%20nvd%20nhd%20njd%20nmd%20nyed%20nynd%20nysd%20nywd%20nced%20ncmd%20ncwd%20ndd%20ohnd%20ohsd%20oked%20oknd%20okwd%20ord%20paed%20pamd%20pawd%20rid%20scd%20sdd%20tned%20tnmd%20tnwd%20txed%20txnd%20txsd%20txwd%20utd%20vtd%20vaed%20vawd%20waed%20wawd%20wvnd%20wvsd%20wied%20wiwd%20wyd%20

Fetching Results: 20batch [00:04,  4.72batch/s, Fetched=20]

Estimated Time Remaining: 8.48 seconds
Fetching: https://www.courtlistener.com/api/rest/v4/search/?court=scotus+ca1+ca2+ca3+ca4+ca5+ca6+ca7+ca8+ca9+ca10+ca11+cadc+cafc+dcd+almd+alnd+alsd+akd+azd+ared+arwd+cacd+caed+cand+casd+cod+ctd+ded+flmd+flnd+flsd+gamd+gand+gasd+hid+idd+ilcd+ilnd+ilsd+innd+insd+iand+iasd+ksd+kyed+kywd+laed+lamd+lawd+med+mdd+mad+mied+miwd+mnd+msnd+mssd+moed+mowd+mtd+ned+nvd+nhd+njd+nmd+nyed+nynd+nysd+nywd+nced+ncmd+ncwd+ndd+ohnd+ohsd+oked+oknd+okwd+ord+paed+pamd+pawd+rid+scd+sdd+tned+tnmd+tnwd+txed+txnd+txsd+txwd+utd+vtd+vaed+vawd+waed+wawd+wvnd+wvsd+wied+wiwd+wyd+gud+nmid+prd+vid&cursor=cz0xMzQxMTI2NzAwMDAwMC4wJnM9Njk0OTg4NDUmdD1yJmQ9MjAyNS0wMS0yOA%3D%3D&filed_after=01%2F01%2F2024&filed_before=01%2F01%2F2025&order_by=dateFiled+desc&q=%22Cybersecurity%22+OR+%22Data+Privacy%22+OR+%22Data+Security%22+OR+%22GDPR%22+OR+%22Data+Protection%22+OR+%22Blockchain%22&type=r


...

Estimated Time Remaining: 12.17 seconds
Fetching: https://www.courtlistener.com/api/rest/v4/search/?court=scotus+ca1+ca2+ca3+ca4+ca5+ca6+ca7+ca8+ca9+ca10+ca11+cadc+cafc+dcd+almd+alnd+alsd+akd+azd+ared+arwd+cacd+caed+cand+casd+cod+ctd+ded+flmd+flnd+flsd+gamd+gand+gasd+hid+idd+ilcd+ilnd+ilsd+innd+insd+iand+iasd+ksd+kyed+kywd+laed+lamd+lawd+med+mdd+mad+mied+miwd+mnd+msnd+mssd+moed+mowd+mtd+ned+nvd+nhd+njd+nmd+nyed+nynd+nysd+nywd+nced+ncmd+ncwd+ndd+ohnd+ohsd+oked+oknd+okwd+ord+paed+pamd+pawd+rid+scd+sdd+tned+tnmd+tnwd+txed+txnd+txsd+txwd+utd+vtd+vaed+vawd+waed+wawd+wvnd+wvsd+wied+wiwd+wyd+gud+nmid+prd+vid&cursor=cz0xMzM4MTQ1OTAwMDAwMC4wJnM9NjgxNjQ0MzYmdD1yJmQ9MjAyNS0wMS0yOA%3D%3D&filed_after=01%2F01%2F2024&filed_before=01%2F01%2F2025&order_by=dateFiled+desc&q=%22Cybersecurity%22+OR+%22Data+Privacy%22+OR+%22Data+Security%22+OR+%22GDPR%22+OR+%22Data+Protection%22+OR+%22Blockchain%22&type=r


Fetching Results: 2660batch [13:30,  3.31batch/s, Fetched=2660]

Estimated Time Remaining: 12.19 seconds
Fetching: https://www.courtlistener.com/api/rest/v4/search/?court=scotus+ca1+ca2+ca3+ca4+ca5+ca6+ca7+ca8+ca9+ca10+ca11+cadc+cafc+dcd+almd+alnd+alsd+akd+azd+ared+arwd+cacd+caed+cand+casd+cod+ctd+ded+flmd+flnd+flsd+gamd+gand+gasd+hid+idd+ilcd+ilnd+ilsd+innd+insd+iand+iasd+ksd+kyed+kywd+laed+lamd+lawd+med+mdd+mad+mied+miwd+mnd+msnd+mssd+moed+mowd+mtd+ned+nvd+nhd+njd+nmd+nyed+nynd+nysd+nywd+nced+ncmd+ncwd+ndd+ohnd+ohsd+oked+oknd+okwd+ord+paed+pamd+pawd+rid+scd+sdd+tned+tnmd+tnwd+txed+txnd+txsd+txwd+utd+vtd+vaed+vawd+waed+wawd+wvnd+wvsd+wied+wiwd+wyd+gud+nmid+prd+vid&cursor=cz0xMzM4MTAyNzAwMDAwMC4wJnM9NjgxNTU3MzkmdD1yJmQ9MjAyNS0wMS0yOA%3D%3D&filed_after=01%2F01%2F2024&filed_before=01%2F01%2F2025&order_by=dateFiled+desc&q=%22Cybersecurity%22+OR+%22Data+Privacy%22+OR+%22Data+Security%22+OR+%22GDPR%22+OR+%22Data+Protection%22+OR+%22Blockchain%22&type=r


Fetching Results: 2680batch [13:38,  2.99batch/s, Fetched=2680]

Estimated Time Remaining: 12.22 seconds
Fetching: https://www.courtlistener.com/api/rest/v4/search/?court=scotus+ca1+ca2+ca3+ca4+ca5+ca6+ca7+ca8+ca9+ca10+ca11+cadc+cafc+dcd+almd+alnd+alsd+akd+azd+ared+arwd+cacd+caed+cand+casd+cod+ctd+ded+flmd+flnd+flsd+gamd+gand+gasd+hid+idd+ilcd+ilnd+ilsd+innd+insd+iand+iasd+ksd+kyed+kywd+laed+lamd+lawd+med+mdd+mad+mied+miwd+mnd+msnd+mssd+moed+mowd+mtd+ned+nvd+nhd+njd+nmd+nyed+nynd+nysd+nywd+nced+ncmd+ncwd+ndd+ohnd+ohsd+oked+oknd+okwd+ord+paed+pamd+pawd+rid+scd+sdd+tned+tnmd+tnwd+txed+txnd+txsd+txwd+utd+vtd+vaed+vawd+waed+wawd+wvnd+wvsd+wied+wiwd+wyd+gud+nmid+prd+vid&cursor=cz0xMzM4MDk0MTAwMDAwMC4wJnM9NjgxNDkxMzgmdD1yJmQ9MjAyNS0wMS0yOA%3D%3D&filed_after=01%2F01%2F2024&filed_before=01%2F01%2F2025&order_by=dateFiled+desc&q=%22Cybersecurity%22+OR+%22Data+Privacy%22+OR+%22Data+Security%22+OR+%22GDPR%22+OR+%22Data+Protection%22+OR+%22Blockchain%22&type=r


In [ ]:

import asyncio
import aiohttp
import nest_asyncio
import pandas as pd
import os
import re
from tqdm.asyncio import tqdm
import time

# Allow nested asyncio in Jupyter if needed
nest_asyncio.apply()

# Function to fetch a single page with retries and exponential backoff
async def fetch_page(session, url, headers, retries=3, backoff_factor=2):
    attempt = 0
    while attempt < retries:
        try:
            async with session.get(url, headers=headers) as response:
                if response.status == 200:
                    return await response.json()
                else:
                    print(f"Error: Failed to fetch {url}, Status code: {response.status}")
                    return None
        except Exception as e:
            print(f"Attempt {attempt + 1} failed for {url}: {e}")
            attempt += 1
            if attempt >= retries:
                print(f"Max retries reached for {url}. Skipping...")
                return None
            # Exponential backoff
            wait_time = backoff_factor ** attempt
            print(f"Retrying in {wait_time} seconds...")
            await asyncio.sleep(wait_time)  # Wait before retrying

# Function to fetch all pages concurrently with batch processing
async def fetch_all_pages_concurrently(api_url, headers, batch_size=20):
    all_results = []
    next_url = api_url
    total_fetched = 0
    failed_urls = []  # List to store URLs that failed to fetch
    start_time = time.time()

    async with aiohttp.ClientSession() as session:
        with tqdm(desc="Fetching Results", unit="batch") as pbar:
            while next_url:
                print(f"Fetching: {next_url}")  # Debugging line
                data = await fetch_page(session, next_url, headers)
                if data and 'results' in data:
                    all_results.extend(data['results'])
                    next_url = data.get('next')  # Update the cursor
                    total_fetched += len(data['results'])
                else:
                    print(f"No results or failed request for {next_url}")
                    break

                pbar.set_postfix({"Fetched": total_fetched})  # Update progress bar
                pbar.update(len(data.get('results', [])))

                # Estimate the time remaining
                elapsed_time = time.time() - start_time
                estimated_time_remaining = (elapsed_time / total_fetched) * (len(data.get('results', [])) * 2)  # Heuristic
                print(f"Estimated Time Remaining: {estimated_time_remaining:.2f} seconds")

    end_time = time.time()
    print(f"\nFinished fetching data in {end_time - start_time:.2f} seconds.")
    print(f"Total records fetched: {total_fetched}")
    return all_results, failed_urls

# Function to retry failed URLs
async def retry_failed_urls(failed_urls, session, headers):
    retry_results = []
    for url in failed_urls:
        result = await fetch_page(session, url, headers)
        if result:
            retry_results.append(result)
    return retry_results

# Function to clean and save the data
def process_and_save_data(all_results, save_path):
    # Create DataFrame
    df = pd.DataFrame(all_results)

    # Function to safely extract the first item from a list
    def safe_get_first(lst):
        return lst[0] if isinstance(lst, list) and len(lst) > 0 else None

    # Expand recap_documents if present
    if 'recap_documents' in df.columns:
        recap_docs = df['recap_documents'].apply(safe_get_first)
        recap_df = pd.json_normalize(recap_docs)
        recap_df.columns = ['recap_' + col for col in recap_df.columns]
        df = pd.concat([df.drop('recap_documents', axis=1), recap_df], axis=1)

    # Remove illegal characters
    ILLEGAL_CHARACTERS_RE = re.compile(r'[\000-\010]|[\013-\014]|[\016-\037]')
    df = df.applymap(lambda x: ILLEGAL_CHARACTERS_RE.sub('', x) if isinstance(x, str) else x)

    # Save to Excel or CSV based on availability
    try:
        df.to_excel(save_path, index=False)
        print(f"\nData saved successfully to {save_path}")
    except Exception as e:
        print(f"Error saving to Excel: {e}. Saving as CSV instead...")
        save_path_csv = save_path.replace('.xlsx', '.csv')
        df.to_csv(save_path_csv, index=False)
        print(f"Data saved successfully to {save_path_csv}")

# Main function to fetch and save data
async def main():
    api_url = "https://www.courtlistener.com/api/rest/v4/search/?q=(%22patent%22%20AND%20(%22artificial%20intelligence%22%20OR%20%22data%22))%20OR%20((%22copyright%22%20AND%20(%22artificial%20intelligence%22%20OR%20%22data%22)))%20OR%20((%22intellectual%20property%22%20AND%20(%22artificial%20intelligence%22%20OR%20%22data%22)))&type=r&order_by=dateFiled%20desc&filed_after=01%2F01%2F2022&court=scotus%20ca1%20ca2%20ca3%20ca4%20ca5%20ca6%20ca7%20ca8%20ca9%20ca10%20ca11%20cadc%20cafc%20dcd%20almd%20alnd%20alsd%20akd%20azd%20ared%20arwd%20cacd%20caed%20cand%20casd%20cod%20ctd%20ded%20flmd%20flnd%20flsd%20gamd%20gand%20gasd%20hid%20idd%20ilcd%20ilnd%20ilsd%20innd%20insd%20iand%20iasd%20ksd%20kyed%20kywd%20laed%20lamd%20lawd%20med%20mdd%20mad%20mied%20miwd%20mnd%20msnd%20mssd%20moed%20mowd%20mtd%20ned%20nvd%20nhd%20njd%20nmd%20nyed%20nynd%20nysd%20nywd%20nced%20ncmd%20ncwd%20ndd%20ohnd%20ohsd%20oked%20oknd%20okwd%20ord%20paed%20pamd%20pawd%20rid%20scd%20sdd%20tned%20tnmd%20tnwd%20txed%20txnd%20txsd%20txwd%20utd%20vtd%20vaed%20vawd%20waed%20wawd%20wvnd%20wvsd%20wied%20wiwd%20wyd%20gud%20nmid%20prd%20vid"

    headers = {"Authorization": "Token xyz}

    # Fetch data using optimized batch fetching
    print("Fetching data...")
    all_results, failed_urls = await fetch_all_pages_concurrently(api_url, headers, batch_size=20)

    # Retry failed URLs if any
    if failed_urls:
        async with aiohttp.ClientSession() as session:
            retry_results = await retry_failed_urls(failed_urls, session, headers)
            all_results.extend(retry_results)

    # Define save path
    desktop_path = os.path.expanduser(r"C:\Users\91909\Desktop\AI")
    save_path = os.path.join(desktop_path, "query10.xlsx")

    # Process and save data
    process_and_save_data(all_results, save_path)

if __name__ == "__main__":
    asyncio.run(main())
